# Exercise: Combine Models

In this exercise you can combine the difference models of py_wake and see the effects in terms of AEP and a flow map

In [12]:
# Importing all required modules
import time
import inspect
from ipywidgets import interact
from py_wake.examples.data.iea37._iea37 import IEA37Site, IEA37_WindTurbines
from py_wake.wind_farm_models.engineering_models import PropagateDownwind, All2AllIterative
from py_wake import deficit_models
from py_wake.deficit_models import DeficitModel
from py_wake.deficit_models import NOJDeficit, GCLDeficit
from py_wake.deficit_models.fuga import FugaDeficit
from py_wake.deficit_models.selfsimilarity import SelfSimilarityDeficit
from py_wake.rotor_avg_models.rotor_avg_model import RotorCenter,CGIRotorAvg, EqGridRotorAvg
from py_wake import superposition_models
from py_wake.superposition_models import SuperpositionModel
from py_wake import turbulence_models
from py_wake.turbulence_models.turbulence_model import TurbulenceModel
from py_wake.examples.data.iea37._iea37 import IEA37Site, IEA37_WindTurbines
import matplotlib.pyplot as plt

In [13]:
# Setup site, wind turbines
site = IEA37Site(16)
wt = IEA37_WindTurbines()
x,y = site.initial_position.T

In [14]:
# setup some utility functions

def get_models(module, baseclass, ignore=set()):
    ignore.add(baseclass)
    return [(v.__name__, v) for v in module.__dict__.values() if inspect.isclass(v) and issubclass(v, baseclass) and v not in ignore]

def run(WindFarmModel, WakeDeficitModel, RotorAvgModel, SuperpositionModel, BlockageDeficitModel, TurbulenceModel):
    if inspect.isclass(WakeDeficitModel):
        WakeDeficitModel = WakeDeficitModel()
    kwargs = {}
    if WindFarmModel==All2AllIterative:
        kwargs['blockage_deficitModel'] = BlockageDeficitModel
    wfm = WindFarmModel(site, wt, wake_deficitModel=WakeDeficitModel, rotorAvgModel=RotorAvgModel, superpositionModel=SuperpositionModel(), 
                        turbulenceModel=TurbulenceModel(),**kwargs)
    t = time.time()
    sim_res = wfm(x,y)
    plt.figure(figsize=(12,8))
    sim_res.flow_map(wd=270).WS_eff.plot()
    print ("Computation time (AEP + flowmap):", time.time()-t)
    plt.title('AEP: %.2fGWh'%(sim_res.aep().sum()))
    wt.plot(x,y,wd=270)

In [15]:
# setup available models
models = {
    'WindFarmModel':[('PropagateDownwind', PropagateDownwind),
                        ('All2AllIterative', All2AllIterative)],
    'WakeDeficitModel':get_models(deficit_models, DeficitModel, {SelfSimilarityDeficit}) + [
                          ('NOJDeficit_ws_eff',NOJDeficit(use_effective_ws=True)), 
                          ('GCLDeficit_ws_eff',GCLDeficit(use_effective_ws=True)),
                          ],
    'RotorAvgModel':[('RotorCenter', RotorCenter()),
                       ('CGIRotorAvg_4', EqGridRotorAvg(4)),
                       ('EqGridRotorAvg_4x4', EqGridRotorAvg(4)),
                       ('EqGridRotorAvg_100x100', EqGridRotorAvg(100))
                      ],
    'SuperpositionModel':get_models(superposition_models, SuperpositionModel),
    'BlockageDeficitModel':[('None',None),
                              ('SelfSimilarityDeficit',SelfSimilarityDeficit()),
                              ('FugaDeficit', FugaDeficit())],
    'TurbulenceModel':get_models(turbulence_models, TurbulenceModel)
         }

In [19]:
_ = interact(run, **models)

interactive(children=(Dropdown(description='WindFarmModel', options=(('PropagateDownwind', <class 'py_wake.win…

Note, the only the All2AllIterative WindFarmModel supports blockage